# Wrangle

In [23]:
# 1. layer: files of size 2140, 1904, 1762, 1857 get merged into a file of size 2514
# 2. layer: the file from the previous step which has the size 2514 gets inner joined with an external file of size 29361 and it results in a file of size 2514
# 3. layer: the file from the previous layer which has the size 2514 gets inner joined with an API and finally has size 1708

import plotly.graph_objects as go
from pathlib import Path

outputpath = Path.cwd().parent / "data" / "assets"

labels = ["Sunshines 2023 (size 2140)", "Sunshines 2022 (size 1904)", "Sunshines 2020 (size 1762)", "Sunshines 2021 (size 1857)", "Sunshines Merged (size 2514)", "CS Rankings (size 29361)", "Sunshines x CS Rankings (size 2514)", "API File (size ≥1708)", "Final File (size 1708)"]

source = [0, 1, 2, 3, 4, 5, 6, 7]
target = [4, 4, 4, 4, 6, 6, 8, 8]
value = [2140, 1904, 1762, 1857, 2514, 29361, 2514, 1708, 1708]

fig = go.Figure(data=[go.Sankey(node=dict(pad=15, thickness=20, line=dict(color="black", width=0.5), label=labels, color="blue"), link=dict(source=source, target=target, value=value))])

fig.update_layout(title_text="Data Joining Process Sankey Diagram", font_size=10)
fig.show()

# store
filepath = outputpath / "wrangle-sankey"
fig.update_layout(
    autosize=False,
    width=1000,
    height=400,
)

# high res
import plotly.io as pio

pio.kaleido.scope.default_scale = 2
pio.kaleido.scope.default_width = 1200
pio.kaleido.scope.default_height = 800
fig.write_image(str(filepath) + ".png")

# Profile

In [53]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm

datapath = Path.cwd().parent / "data" / "sunshines-v4.csv"
df = pd.read_csv(datapath)

#
# infer features
#


def get_latest_totalcomp(row):
    years = ["2023", "2022", "2021", "2020"]
    for year in years:
        salary_col = f"salary_{year}"
        benefits_col = f"benefits_{year}"
        if salary_col in row.index and benefits_col in row.index:  # fall back to previous year if current year is missing
            if pd.notna(row[salary_col]) and pd.notna(row[benefits_col]):  # check if both present
                return row[salary_col] + row[benefits_col]  # get total comp
    print(f"warning: {row['name']}")
    return pd.NA


def get_latest_role(row):
    years = ["2023", "2022", "2021", "2020"]
    for year in years:
        role_col = f"role_{year}"
        if role_col in row.index:  # fall back to previous year if current year is missing
            if pd.notna(row[role_col]):  # check if present
                return row[role_col]
    print(f"warning: {row['name']}")
    return pd.NA


def get_latest_role_cluster(row):
    years = ["2023", "2022", "2021", "2020"]
    for year in years:
        role_col = f"role_cluster_{year}"
        if role_col in row.index:  # fall back to previous year if current year is missing
            if pd.notna(row[role_col]):  # check if present
                return row[role_col]
    print(f"warning: {row['name']}")
    return pd.NA


df["latest_totalcomp"] = df.apply(get_latest_totalcomp, axis=1)
df["latest_role"] = df.apply(get_latest_role, axis=1)
df["latest_role_cluster"] = df.apply(get_latest_role_cluster, axis=1)

df["perf_combined"] = df["paper_count"] + df["citation_count"] + df["h_index"]
for year in range(2020, 2024):
    df[f"totalcomp_{year}"] = df[f"salary_{year}"] + df[f"benefits_{year}"]

print(df.dtypes)
df.head()


#
# correlation heatmap
#


# IDEA: demographics
# male female ratio
# male female ratio per role

# IDEA: career progression (temporal)
# - change in roles over time, ranking roles

# IDEA: compensation trends (temporal)
# - visualize num employees, salaries, benefits, num papers, citations, h_index, perf_combined - over time

# IDEA: performance (correlation)
# what influences latest_totalcomp the most?
# correlate using a heatmap:
# - sex
# - paper_count
# - citation_count
# - h_index
# - latest_role
# - perf_combined (combined)

name                    object
sex                     object
paper_count              int64
citation_count           int64
h_index                  int64
role_2020               object
role_cluster_2020      float64
salary_2020            float64
benefits_2020          float64
role_2021               object
role_cluster_2021      float64
salary_2021            float64
benefits_2021          float64
role_2022               object
role_cluster_2022      float64
salary_2022            float64
benefits_2022          float64
role_2023               object
role_cluster_2023      float64
salary_2023            float64
benefits_2023          float64
latest_totalcomp       float64
latest_role             object
latest_role_cluster    float64
perf_combined            int64
totalcomp_2020         float64
totalcomp_2021         float64
totalcomp_2022         float64
totalcomp_2023         float64
dtype: object


,name,sex,paper_count,citation_count,h_index,role_2020,role_cluster_2020,salary_2020,benefits_2020,role_2021,...,salary_2023,benefits_2023,latest_totalcomp,latest_role,latest_role_cluster,perf_combined,totalcomp_2020,totalcomp_2021,totalcomp_2022,totalcomp_2023
0,Yousra Aafer,F,34,2998,18,Assistant Professor,3.0,148935.04,158.96,Assistant Professor,...,160442.56,276.08,160718.64,Assistant Professor,3.0,3050,149094.00,154313.40,159488.20,160718.64
1,Eihab Abdel-rahman,M,197,5193,31,Professor,3.0,195256.60,215.48,Professor,...,221699.84,370.76,222070.60,Professor,3.0,5421,195472.08,206567.72,210558.60,222070.60
2,Ishan S. Abeywardena,M,37,144,6,Associate Director Systems,19.0,125246.72,334.00,Associate Director Systems,...,142080.64,550.72,142631.36,Associate Director Systems,19.0,187,125580.72,130435.29,134382.97,142631.36
3,Hossein Abouee Mehrizi,M,31,450,13,Associate Professor,3.0,175698.40,293.66,Associate Professor,...,209521.60,755.32,210276.92,Professor,3.0,494,175992.06,208375.44,207523.76,210276.92
4,John Abraham,M,383,9432,47,NaN,NaN,NaN,NaN,Continuing Lecturer and Manager of Internation...,...,134059.52,875.24,134934.76,Academic Dean Continuing Lecturer and Manager ...,10.0,9862,NaN,105171.84,115881.00,134934.76
